# RAG Evaluation Without Ground Truth — Experiments

**Mục tiêu:** Thử nghiệm các framework evaluation không cần `expected_answer` và `ground_truth_context`

## Vấn đề

Hệ thống hiện tại phụ thuộc vào:
- ❌ `expected_answer` — không có sẵn trong thực tế
- ❌ `ground_truth_context` — không có vì RAG tự động retrieve

## Giải pháp

Test 3 frameworks:
1. **DeepEval** — LLM-as-Judge (no ground truth mode)
2. **RAGAS** — RAG-specific metrics
3. **OpenRAG-Eval** (Optional) — Research approach

## Workflow

1. Load test cases
2. Generate answers from RAG API
3. Evaluate with each framework
4. Meta-evaluate: So sánh consistency
5. Human-in-the-loop validation

---

## 📦 Setup & Installation

Cài đặt các thư viện cần thiết

In [11]:
!pip install ragas datasets langchain langchain-community langchain-openai
!pip install deepeval pandas matplotlib seaborn scipy

In [12]:
# Install packages nếu cần (uncomment)
# !pip install ragas datasets langchain langchain-community langchain-openai
# !pip install deepeval pandas matplotlib seaborn scipy

import os, sys, warnings, json, pandas as pd, numpy as np
from typing import Dict, List, Any
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append(os.path.abspath('.'))

# Load .env và kiểm tra API key
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Bạn chưa đặt OPENAI_API_KEY trong file .env")

print("✅ Setup complete!")

✅ Setup complete!


## 1️⃣ Load Test Cases

Load và explore test cases từ `data/testcases.json`

In [13]:
import json
from datasets import Dataset

# 1. Load Ground Truth Map
with open('../data/pipeline/query.json', 'r', encoding='utf-8') as f:
    standard_data = json.load(f)

# Chuẩn hóa key: viết thường và bỏ khoảng trắng thừa
gt_map = {item['original'].strip(): item['answer'] for item in standard_data}

# 2. Load Pipeline Results từ file JSON của bạn
# Thay đổi đường dẫn file cho đúng với file bạn đang dùng
file_path = '../data/pipeline/ragas_evaluation_hybrid_20260111_192121.json'
with open(file_path, 'r', encoding='utf-8-sig') as f:
    results_data = json.load(f)

ragas_input = {"question": [], "answer": [], "contexts": [], "ground_truth": []}

# Xử lý trường hợp results_data là một list hoặc một dict
entries = results_data if isinstance(results_data, list) else [results_data]

for item in entries:
    # Truy cập vào danh sách kết quả (tùy cấu trúc file của bạn)
    res_list = item.get('results', []) if isinstance(item, dict) else []
    
    for r in res_list:
        q_raw = r.get('question', '').strip()
        
        if q_raw in gt_map:
            ragas_input["question"].append(q_raw)
            ragas_input["answer"].append(r.get('answer', ''))
            ragas_input["contexts"].append(r.get('contexts', []))
            ragas_input["ground_truth"].append(gt_map[q_raw])

ragas_dataset = Dataset.from_dict(ragas_input)
print(f"✅ Đã chuẩn bị xong {len(ragas_dataset)} test cases.")

if len(ragas_dataset) == 0:
    print("⚠️ CẢNH BÁO: Vẫn chưa tìm thấy test case nào. Hãy kiểm tra lại key 'question' trong file JSON.")

# Kiểm tra nếu có dữ liệu
if len(ragas_dataset) > 0:
    print(f"--- KIỂM TRA DỮ LIỆU CÂU ĐẦU TIÊN ---")
    # In câu hỏi
    print(f"❓ Question: {ragas_input['question'][20]}")
    
    # In câu trả lời từ RAG
    print(f"🤖 Answer: {ragas_input['answer'][20]}")
    
    # In Ground Truth để đối chiếu
    print(f"🎯 Ground Truth: {ragas_input['ground_truth'][20]}")
    
    # In số lượng context đã tìm thấy
    print(f"📄 Contexts count: {len(ragas_input['contexts'][20])}")
    print("-" * 40)
else:
    print("❌ Không có dữ liệu để hiển thị. Vui lòng kiểm tra lại logic so khớp câu hỏi.")

✅ Đã chuẩn bị xong 35 test cases.
--- KIỂM TRA DỮ LIỆU CÂU ĐẦU TIÊN ---
❓ Question: Bố cục làm đồ án tốt nghiệp như nào nhỉ?
🤖 Answer: Dựa trên thông tin được cung cấp, bố cục làm đồ án tốt nghiệp như sau:

*   **MỞ ĐẦU:** Trình bày lí do chọn đề tài, mục đích, đối tượng và phạm vi nghiên cứu.
*   **TỔNG QUAN:** Phân tích đánh giá các hướng nghiên cứu đã có của các tác giả trong và ngoài nước liên quan đến đề tài; nêu những vấn đề còn tồn tại; chỉ ra những vấn đề mà đề tài cần tập trung, nghiên cứu giải quyết.
*   **NGHIÊN CỨU THỰC NGHIỆM HOẶC LÍ THUYẾT:** Trình bày cơ sở lí thuyết, lí luận, giả thiết khoa học và phương pháp nghiên cứu đã được sử dụng trong đồ án.
*   **TRÌNH BÀY, ĐÁNH GIÁ BÀN LUẬN VỀ KẾT QUẢ:** Mô tả ngắn gọn công việc nghiên cứu khoa học đã tiến hành, các kết quả nghiên cứu khoa học hoặc kết quả thực nghiệm. Đối với các đề tài ứng dụng có kết quả là sản phẩm phần mềm phải có hồ sơ thiết kế, cài đặt, ... theo các mô hình đã học (UML, ...).
*   **KẾT LUẬN:** Trình bày 

## 3️⃣ RAGAS Evaluation (No Ground Truth)

RAGAS có các metrics không cần `expected_answer`:
- **Faithfulness** — Answer có faithful với retrieved context không?
- **Answer Relevancy** — Answer có relevant với question không?

Cả 2 metrics này đều reference-free!

In [14]:
! pip install langchain_openai


In [15]:
# Setup RAGAS with Ollama (local LLM)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Initialize Ollama LLM for RAGAS
ollama_llm = ChatOpenAI(model="gpt-4o-mini")
ollama_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

ragas_llm = LangchainLLMWrapper(ollama_llm)
ragas_embeddings = LangchainEmbeddingsWrapper(ollama_embeddings)

In [ ]:
# ---------------------------------------------------
# 2️⃣ Import các metric async-compatible
# ---------------------------------------------------

from ragas.metrics import AnswerCorrectness, AnswerRelevancy, Faithfulness, ContextRelevance, ContextRecall
import pandas as pd

metrics_dict = {
    "AnswerCorrectness": AnswerCorrectness(llm=ragas_llm),
    "AnswerRelevancy": AnswerRelevancy(embeddings=ragas_embeddings),
    "Faithfulness": Faithfulness(llm=ragas_llm),
    "ContextRelevance": ContextRelevance(llm=ragas_llm),
    "ContextRecall": ContextRecall()
}

summary_scores = {}

for name, metric in metrics_dict.items():
    print(f"\n🚀 Running metric: {name} ...")
    
    result = evaluate(
        dataset=ragas_dataset,
        metrics=[metric],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        batch_size=5,
        show_progress=True
    )
    
    score = list(result._scores_dict.values())[0]
    summary_scores[name] = score

df_scores = pd.DataFrame([summary_scores])
print("\n✅ Summary scores preview:")
print(df_scores)



🚀 Running metric: AnswerCorrectness ...


Evaluating:   0%|          | 0/35 [00:00<?, ?it/s]Exception raised in Job[4]: TimeoutError()
Exception raised in Job[2]: TimeoutError()
Exception raised in Job[3]: TimeoutError()
Evaluating:  14%|█▍        | 5/35 [03:03<18:21, 36.71s/it]Exception raised in Job[7]: TimeoutError()
Exception raised in Job[5]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[6]: TimeoutError()
Exception raised in Job[9]: TimeoutError()
Evaluating:  29%|██▊       | 10/35 [06:54<17:35, 42.24s/it]

In [ ]:
import numpy as np

# Tạo dict để lưu trung bình
avg_scores = {}

for col in df_scores.columns:
    avg_scores[col] = np.mean(df_scores[col][0])  # df_scores[col][0] là list score

# Xuất kết quả
print("\n✅ Average score for each metric:")
for metric, avg in avg_scores.items():
    print(f"{metric}: {avg:.4f}")



✅ Average score for each metric:
AnswerCorrectness: 0.3381
AnswerRelevancy: 0.3603
Faithfulness: 0.3103
ContextRelevance: 0.6357
ContextRecall: 0.4000


In [ ]:
import pandas as pd
import csv

# Xuất df_scores với UTF-8 và quoting
df_scores.to_csv(
    "hybrid_scores.csv",      # tên file xuất ra
    index=False,               # không xuất cột index
    encoding="utf-8-sig",      # utf-8-sig giúp Excel đọc đúng tiếng Việt
    quoting=csv.QUOTE_ALL      # đặt tất cả giá trị trong dấu ngoặc kép
)

print("✅ df_scores đã được lưu vào summary_scores.csv với UTF-8 và quoting")


✅ df_scores đã được lưu vào summary_scores.csv với UTF-8 và quoting


In [ ]:
import pandas as pd
import ast
import csv

# 1. Đọc file
df = pd.read_csv("hybrid_scores.csv")

# 2. Hàm dọn dẹp dữ liệu (giữ nguyên logic cũ nhưng thêm xử lý lỗi)
def clean_and_parse(s):
    if isinstance(s, list): return s
    if not isinstance(s, str) or s.lower() == 'nan': return []
    s = s.replace("np.float64(", "").replace(")", "")
    try:
        return ast.literal_eval(s)
    except:
        return []

# 3. Cách mới: Bung từng cột (Explode) một cách độc lập để tránh lỗi index
exploded_cols = {}
for col in df.columns:
    # Lấy dữ liệu dòng đầu tiên, parse thành list, rồi chuyển thành Series
    data_list = clean_and_parse(df[col].iloc[0])
    exploded_cols[col] = pd.Series(data_list)

# Tạo DataFrame mới từ các cột đã bung
df_flat = pd.DataFrame(exploded_cols)

# 4. Xuất file CSV
output_file = "hybrid_results_flat.csv"
df_flat.to_csv(
    output_file, 
    index=False, 
    encoding="utf-8-sig",
    quoting=csv.QUOTE_NONNUMERIC
)

print(f"✅ Đã xử lý xong! File '{output_file}' đã sẵn sàng.")
print(df_flat)

✅ Đã xử lý xong! File 'naive_results_flat.csv' đã sẵn sàng.
    AnswerCorrectness  AnswerRelevancy  Faithfulness  ContextRelevance  \
0            0.161569         0.338033      0.166667              0.75   
1            0.165271         0.306052      0.000000              1.00   
2            0.715650         0.337067      0.000000              1.00   
3            0.414763         0.761386      0.000000              0.00   
4            0.773079         0.334505      1.000000              1.00   
5            0.158986         0.295671      1.000000              1.00   
6            0.175921         0.325610      1.000000              1.00   
7            0.711161         0.182434      0.000000              0.25   
8            0.166561         0.474354      0.000000              0.25   
9            0.138070         0.373547      0.000000              0.75   
10           0.172681         0.269540      0.357143              0.75   
11           0.775811         0.257310      0.222222